In [1]:
import time

import torch
import torchvision

In [2]:
model = torchvision.models.resnet50()
model.eval()

# ダミー画像を用意
input_image = torch.ones((32, 3, 224, 224))

# ウォームアップのため一度走らせる
with torch.no_grad():
    output = model(input_image)

# モデルの計算時間を計測（FP 32）
start_time = time.time()
with torch.no_grad():
    output = model(input_image)
end_time = time.time()

computation_time = end_time - start_time
print(computation_time)
# -> AWS EC2 c7i.4xlarge (Xeon Platinum 8488C, 8 コア 16 vCPU) で 0.793 秒

0.7010986804962158


In [3]:
# BF 16 に変換
model = model.bfloat16()
input_image = input_image.bfloat16()

with torch.no_grad():
    output = model(input_image)

# モデルの計算時間を計測（BF 16）
start_time = time.time()
with torch.no_grad():
    output = model(input_image)
end_time = time.time()

computation_time = end_time - start_time
print(computation_time)
# -> AWS EC2 c7i.4xlarge (Xeon Platinum 8488C, 8 コア 16 vCPU) で 0.432 秒

0.365720272064209


In [4]:
import intel_extension_for_pytorch as ipex

In [5]:
model = ipex.optimize(model, dtype=torch.bfloat16)

with torch.cpu.amp.autocast():
    model(input_image)

with torch.cpu.amp.autocast():
    start_time = time.time()
    model(input_image)
    end_time = time.time()

computation_time = end_time - start_time
print(computation_time)
# -> AWS EC2 c7i.4xlarge (Xeon Platinum 8488C, 8 コア 16 vCPU) で 0.137 秒

0.12478256225585938
